In [3]:
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import smote_variants as sv
from tqdm import tqdm
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import os

In [2]:
df1=pd.read_csv("dataset/datawithoglist.csv")
df2=pd.read_csv("dataset/datawithcleanog.csv")
df3=pd.read_csv("dataset/datawithsmote.csv")
df4=pd.read_csv("dataset/datawithcossmote.csv")
df1=df1[['body','description','title','url','cleaned_website_text','words matched','category']]
df1.rename(columns = {'category':'cat_og'}, inplace = True)
df1.rename(columns = {'words matched':'words_og'}, inplace = True)
df2=df2[['body','description','title','url','cleaned_website_text','words matched','category']]
df2.rename(columns = {'category':'cat_ogclean'}, inplace = True)
df2.rename(columns = {'words matched':'words_ogclean'}, inplace = True)
df3=df3[['body','description','title','url','cleaned_website_text','words matched','category']]
df3.rename(columns = {'category':'cat_smote'}, inplace = True)
df3.rename(columns = {'words matched':'words_smote'}, inplace = True)
df4=df4[['body','description','title','url','cleaned_website_text','words matched','category']]
df4.rename(columns = {'category':'cat_cossmote'}, inplace = True)
df4.rename(columns = {'words matched':'words_cossmote'}, inplace = True)

In [3]:
df1['cat_og'].value_counts()

drug        11745
non drug     8489
Name: cat_og, dtype: int64

In [4]:
df2['cat_ogclean'].value_counts()

drug        18466
non drug     1768
Name: cat_ogclean, dtype: int64

In [5]:
df3['cat_smote'].value_counts()

drug        20091
non drug      143
Name: cat_smote, dtype: int64

In [6]:
df4['cat_cossmote'].value_counts()

drug        17565
non drug     2669
Name: cat_cossmote, dtype: int64

In [7]:
cat_ogclean=list(df2['cat_ogclean'])
cat_smote=list(df3['cat_smote'])
cat_cossmote=list(df4['cat_cossmote'])
words_ogclean=list(df2['words_ogclean'])
words_smote=list(df3['words_smote'])
words_cossmote=list(df4['words_cossmote'])
df1['words_ogclean']=words_ogclean
df1['cat_ogclean']=cat_ogclean
df1['words_smote']=words_smote
df1['cat_smote']=cat_smote
df1['words_cossmote']=words_cossmote
df1['cat_cossmote']=cat_cossmote
df12=df1.drop_duplicates(keep="first",subset=['url'])
df12

,body,description,title,url,cleaned_website_text,words_og,cat_og,words_ogclean,cat_ogclean,words_smote,cat_smote,words_cossmote,cat_cossmote
0,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",Cocaine Drugs,http://fc3ryhftqfgwyroq7pt63f7jif4jknfrmd3pbdw...,cookie javascript trace protect privacy home q...,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug,"['privacy', 'g', 'mix', 'g', 'mg', 'drugs']",drug,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug
1,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",Cannabis,http://r3q6nxpmfdxhzz3oumaseyfr5qujvg4cayzvj2y...,cookie javascript trace protect privacy home q...,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug,"['privacy', 'g', 'mix', 'g', 'mg', 'drugs']",drug,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug
2,\n \n \n \n \n \n \n \n \n \n \n drugshop@dnmx...,"Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.","Buy Drugs - Ecstasy, LSD, Meth, Cocaine, Canna...",http://7nroowrqwdgbyajwrynxtypkdi46f22akwki73e...,home catalog question answer cart contact escr...,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug,"['g', 'add', 'mix', 'add', 'g', 'add', 'mg', '...",drug,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug
3,\n \n \n \n \n \n \n \n \n \n \n drugsshopper...,Drugs Shop - buy cannabis best marijuana hemp.,Cannabis - Seeds Cannabis - Drugs Shop hash,http://hpcphv5q3lhiebh6c765lslihne4tn6ui5dnd63...,home catalog question answer cart contact cate...,"['cannabis', 'cannabis', 'kush', 'cannabis', '...",drug,"['cannabis', 'cannabis', 'kush', 'cannabis', '...",drug,"['g', 'add', 'g', 'add', 'g', 'add', 'g', 'add...",drug,"['cannabis', 'cannabis', 'kush', 'cannabis', '...",drug
4,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",DRUGS,http://lnn6isw6tkbmplxe7eisjpf7nw6mmbwc7wjjuyh...,cookie javascript trace protect privacy home q...,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug,"['privacy', 'g', 'mix', 'g', 'mg', 'drugs']",drug,"['cannabis', 'cannabis', 'cocaine', 'ecstasy',...",drug
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20229,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,"On the Internet, Forever-Mutating Debates\n",https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,january internet forever mutating debates rela...,['weed'],drug,"['weed', 'special', 'system']",drug,"['economy', 'industry', 'dog', 'remember', 'in...",drug,"['weed', 'crime', 'take']",drug
20230,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,1997 Technology Highlights,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,january technology highlights related articles...,"['soap', 'peace']",drug,"['soap', 'love', 'peace', 'system', 'blue', 'b...",drug,"['economy', 'industry', 'win', 'like', 'decade...",drug,"['soap', 'take', 'take', 'peace', 'product']",drug
20231,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,Stock Volatility,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,credit new york times article home site index ...,[],non drug,[],non drug,['weather'],drug,[],non drug
20232,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,Wall Street,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,volatile day volatile year wall street clean n...,[],non drug,[],non drug,"['day', 'weather']",drug,[],non drug


In [8]:
test1=df1.query('cat_og=="non drug" & cat_ogclean=="non drug"')
len(test1)

1767

In [9]:
test2=df1.query('cat_og=="drug" & cat_ogclean=="non drug"')
len(test2)

1

In [10]:
test3=df1.query('cat_og=="non drug" & cat_ogclean=="drug"')
len(test3)

6722

In [11]:
#new testcases

In [12]:
t1=df1.query('cat_og=="non drug" & cat_cossmote=="non drug"')
len(t1)

2669

In [13]:
t2=df1.query('cat_og=="drug" & cat_cossmote=="non drug"')
len(t2)

0

In [19]:
t3=df1.query('cat_og=="non drug" & cat_cossmote=="drug"')
len(t3)

5820

In [17]:
t3.head(5)

,body,description,title,url,cleaned_website_text,words_og,cat_og,words_ogclean,cat_ogclean,words_smote,cat_smote,words_cossmote,cat_cossmote
9,OnionLinks Short .onion v2 websites will stop ...,NaN,OnionLinks,http://jaz45aabn5vkemy4jkg4mi4syheisqn2wn2n4fs...,onionlinks short v2 website stop work october ...,[],non drug,[],non drug,"['drug', 'privacy', 'dark', 'dark', 'drugs', '...",drug,"['drug', 'counterfeit', 'drugs', 'drug']",drug
22,OnionLinks Short .onion v2 websites will stop ...,NaN,OnionLinks,http://c5xoy22aadb2rqgw3jh2m2irmu563evukqqddu5...,onionlinks short v2 website stop work october ...,[],non drug,[],non drug,"['drug', 'privacy', 'dark', 'dark', 'drugs', '...",drug,"['drug', 'counterfeit', 'drugs', 'drug']",drug
37,OnionLinks Short .onion v2 websites will stop ...,NaN,OnionLinks,http://ez37hmhem2gh3ixctfeaqn7kylal2vyjqsedkzh...,onionlinks short v2 website stop work october ...,[],non drug,[],non drug,"['drug', 'privacy', 'dark', 'dark', 'drugs', '...",drug,"['drug', 'counterfeit', 'drugs', 'drug']",drug
46,OnionLinks Short .onion v2 websites will stop ...,NaN,OnionLinks,http://gkcns4d3453llqjrksxdijfmmdjpqsykt6misgo...,onionlinks short v2 website stop work october ...,[],non drug,[],non drug,"['drug', 'privacy', 'dark', 'dark', 'drugs', '...",drug,"['drug', 'counterfeit', 'drugs', 'drug']",drug
60,OnionLinks Short .onion v2 websites will stop ...,NaN,OnionLinks,http://xjfbpuj56rdazx4iolylxplbvyft2onuerjeiml...,onionlinks short v2 website stop work october ...,[],non drug,[],non drug,"['drug', 'privacy', 'dark', 'dark', 'drugs', '...",drug,"['drug', 'counterfeit', 'drugs', 'drug']",drug


In [47]:
import ast
l1=[]
l2=[]
urls=['http://c5xoy22aadb2rqgw3jh2m2irmu563evukqqddu5zjandunaimzaye5id.onion/',
      'http://gkcns4d3453llqjrksxdijfmmdjpqsykt6misgojxlhsnpivtl3uwhqd.onion/',
     'http://porf65zpwy2yo4sjvynrl4eylj27ibrmo5s2bozrhffie63c7cxqawid.onion/',
     'http://4p6i33oqj6wgvzgzczyqlueav3tz456rdu632xzyxbnhq4gpsriirtqd.onion/',
     'http://gn74rz534aeyfxqf33hqg6iuspizulmvpd7zoyz7ybjq4jo3whkykryd.onion/']
x=df1.loc[df1['url']==urls[0]] 
l1.append(set(ast.literal_eval(x['words_og'].iloc[1])))
l2.append(set(ast.literal_eval(x['words_cossmote'].iloc[1])))
x=df1.loc[df1['url']==urls[1]] 
l1.append(set(ast.literal_eval(x['words_og'].iloc[2])))
l2.append(set(ast.literal_eval(x['words_cossmote'].iloc[2])))
x=df1.loc[df1['url']==urls[2]] 
l1.append(set(ast.literal_eval(x['words_og'].iloc[1])))
l2.append(set(ast.literal_eval(x['words_cossmote'].iloc[1])))
x=df1.loc[df1['url']==urls[3]] 
l1.append(set(ast.literal_eval(x['words_og'].iloc[2])))
l2.append(set(ast.literal_eval(x['words_cossmote'].iloc[2])))
x=df1.loc[df1['url']==urls[4]] 
l1.append(set(ast.literal_eval(x['words_og'].iloc[0])))
l2.append(set(ast.literal_eval(x['words_cossmote'].iloc[0])))

In [48]:
l1

[{'acid', 'cocaine', 'crack', 'heroin', 'lsd', 'mdma'},
 {'cannabis', 'cocaine', 'crystal', 'crystal meth', 'ecstasy', 'lsd', 'meth'},
 {'cannabis', 'cocaine', 'crystal', 'crystal meth', 'ecstasy', 'lsd', 'meth'},
 {'cannabis', 'cocaine', 'crystal', 'crystal meth', 'ecstasy', 'lsd', 'meth'},
 {'cannabis', 'cheese', 'chronic', 'hash', 'kush'}]

In [49]:
l2

[{'acid', 'cocaine', 'crack', 'drug', 'heroin', 'lsd', 'mdma', 'product'},
 {'cannabis',
  'cocaine',
  'crystal',
  'crystal meth',
  'drugs',
  'ecstasy',
  'lsd',
  'meth',
  'pills'},
 {'cannabis',
  'cocaine',
  'crystal',
  'crystal meth',
  'drugs',
  'ecstasy',
  'lsd',
  'meth',
  'pills'},
 {'cannabis',
  'cocaine',
  'crystal',
  'crystal meth',
  'drugs',
  'ecstasy',
  'lsd',
  'meth',
  'pills'},
 {'cannabis', 'cheese', 'chronic', 'hash', 'kush', 'product'}]